In [1]:
\c 5 100
h:`$":tpr-mscf-kx.tepper.cmu.edu:5000:mscf2023:LZ8AByXHDaEJdTim"

In [2]:
/%python
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import OptimizeResult
plt.style.use('default')

### Load Data

In [17]:
loaddata:{[s;c;sd;ed]
 / YOUR CODE HERE
 t:select date,sym,time,price,size from trade where date within (sd;ed), sym in s, cond in c;
 t}

show t_save:h (loaddata;`AAPL`NVDA`BABA;" ";2020.02.01;2020.02.05)

date       sym  time                 price  size
------------------------------------------------
2020.02.03 AAPL 0D09:30:00.104222076 304.25 200 
2020.02.03 AAPL 0D09:30:00.104222222 304.25 300 
2020.02.03 AAPL 0D09:30:00.115302599 304.16 100 
2020.02.03 AAPL 0D09:30:00.420630361 304.18 120 
2020.02.03 AAPL 0D09:30:00.425902385 304.18 100 
..


In [18]:
show t: update return:fills 0^log(prev price)%price from t_save

date       sym  time                 price  size return      
-------------------------------------------------------------
2020.02.03 AAPL 0D09:30:00.104222076 304.25 200  0           
2020.02.03 AAPL 0D09:30:00.104222222 304.25 300  0           
2020.02.03 AAPL 0D09:30:00.115302599 304.16 100  0.0002958531
2020.02.03 AAPL 0D09:30:00.420630361 304.18 120  -6.57527e-05
2020.02.03 AAPL 0D09:30:00.425902385 304.18 100  0           
..


In [19]:
t

date       sym  time                 price  size return      
-------------------------------------------------------------
2020.02.03 AAPL 0D09:30:00.104222076 304.25 200  0           
2020.02.03 AAPL 0D09:30:00.104222222 304.25 300  0           
2020.02.03 AAPL 0D09:30:00.115302599 304.16 100  0.0002958531
2020.02.03 AAPL 0D09:30:00.420630361 304.18 120  -6.57527e-05
2020.02.03 AAPL 0D09:30:00.425902385 304.18 100  0           
..


In [20]:
train_data: select from t where date < 2020.02.04
test_data: select from t where date >= 2020.02.04

In [22]:
test_data

date       sym  time                 price  size return       
--------------------------------------------------------------
2020.02.04 AAPL 0D09:30:00.095982671 315.47 100  -0.2719674   
2020.02.04 AAPL 0D09:30:00.095984948 315.48 400  -3.169823e-05
2020.02.04 AAPL 0D09:30:00.170162286 315.48 185  0            
2020.02.04 AAPL 0D09:30:00.416661690 315.5  100  -6.339345e-05
2020.02.04 AAPL 0D09:30:00.416689735 315.5  100  0            
..


### Calculate predictability for k stocks
1. For seconds, minutes, hours frequency over D days, get MSE_i for stock i
2. return (k, n/D, 3)
3. Prediction model (average)
   - Autoregressive moving average model (ARIMA)
   - LSTM

In [39]:
get_features:{[t]
 // return the average price and sum of trading sizes within the given time step
 x_sec: select price:(sum (price*size))%sum size,size:sum size by time.second, sym, date from t;
 x_min: select price:(sum (price*size))%sum size,size:sum size by time.minute, sym, date from t;
 x_hr: select  price:(sum (price*size))%sum size,size:sum size by time.hh, sym, date from t;
 dfs: (x_sec;x_min;x_hr);
 dfs}

dfs: get_features[test_data]
x_sec: dfs[0];
x_min: dfs[1]
x_hr: dfs[2]

In [40]:
x_hr

hh sym  date      | price    size   
------------------| ----------------
9  AAPL 2020.02.04| 315.3055 3604668
9  AAPL 2020.02.05| 322.9354 3454343
9  BABA 2020.02.04| 221.6647 2416575
9  BABA 2020.02.05| 222.3097 2122567
9  NVDA 2020.02.04| 245.7631 660613 
..


In [40]:
liquidty:{[t, sym, wsize]
 /t: data frame, sym: symbol list, wsize: number of days in the function 
 / add one more column in dataframe for each symbol on its amihub illiquidity
 amihud:avg abs[log close%prev close]%price,dvol:1e-9*avg price by sym from daily;
 amihud}
 

[0;31mparse error[0m: [0;31mtype[0m